In [15]:
import numpy as np #we use numpy instead of python list since it is faster in computation

# import module
from tabulate import tabulate
  
# assign data
mydata = [{"Nikhil", "Delhi"}, 
          {"Ravi", "Kanpur"}, 
          {"Manish", "Ahmedabad"}, 
          {"Prince", "Bangalore"}]
  
# create header
head = ["Name", "City"]
  
# display table
print(tabulate(np.zeros([3,3]), tablefmt="grid"))


+---+---+---+
| 0 | 0 | 0 |
+---+---+---+
| 0 | 0 | 0 |
+---+---+---+
| 0 | 0 | 0 |
+---+---+---+
0


In [18]:
from enum import Enum
from random import randint, choice
from copy import copy
import numpy as np #we use numpy instead of python list since it is faster in computation
from tabulate import tabulate

class Point2D:
    def __init__(self,x_value,y_value) -> None:
        self.x_value = x_value
        self.y_value = y_value 

class Action(Enum):
    def __str__(self):
        return self.name
    Left = 1
    Right = 2
    Up = 3
    Down = 4

class GameEnvironment():
    def __init__(self):
        self.__initial_state = np.zeros([3,3],int)
        self.__state = self.__initial_state
        self.__state[-1][-1], self.__state[-1][-2]= 2,2
        self.player_turn = True
    
    def reset(self):
        self.__state = self.__initial_state
        return self.__state
       
    def  is_possible_action(self, action: Action, first_point: tuple, second_point: tuple) -> bool:
        if self.__state[first_point[0]][first_point[1]] == self.__state[second_point[0]][second_point[1]] == 0:
            return False
        if self.__state[first_point[0]][first_point[1]] != self.__state[second_point[0]][second_point[1]]:
            return False
        if action == Action.Down and first_point[1] != second_point[1] and first_point[0] >= second_point[0]:
            return False
        if action == Action.Up and first_point[1] != second_point[1] and first_point[0] <= second_point[0]:
            return False
        if action == Action.Left and first_point[0] != second_point[0] and first_point[1] <= second_point[1]:
            return False
        if action == Action.Right and first_point[0] != second_point[0] and first_point[1] >= second_point[1]:
            return False
        
    def __calculate_transition(self, action: Action, first_point: tuple, second_point: tuple):
        if self.__state[first_point[0]][first_point[1]] != self.__state[second_point[0]][second_point[1]]:
            return self.__state
        if action == Action.Down:
            if first_point[1] != second_point[1] :
                return self.__state
            else:
                pass

        if self.is_done():
            return self.__state
        # let the opponent make a random move
        opponent_action = choice(self.get_possible_actions())
        self.__state[opponent_action] = O
        return self.__state
      
    def step(self, action):
        self.__state = self.__calculate_transition(action)  # state after action
        observation = self.__state  # environment is fully observable
        done = self.is_done()
        reward = self.get_reward(self.__state)
        info = {}  # optional debug info
        return observation, done, reward, info

    def render(self):
        print(tabulate(self.__state,tablefmt="grid"))
        
    #=========================================================
    # public functions for agent to calculate optimal policy
    #=========================================================
    
    def get_possible_states(self):
        return self.__possible_states
    
    def get_possible_actions(self, state=None): # an action should be a tuple that contains (coord_1, coord_2, direction)
        if state is None:
            state = self.__state
        return [n for n in range(9) if state[n] == E]

    def is_done(self, state=None):
        if state is None:
            state = self.__state
        if E not in state:
            return True
        if state[0] == state[1] == state[2] != E:
            return True
        if state[3] == state[4] == state[5] != E:
            return True
        if state[6] == state[7] == state[8] != E:
            return True
        if state[0] == state[3] == state[6] != E:
            return True
        if state[1] == state[4] == state[7] != E:
            return True
        if state[2] == state[5] == state[8] != E:
            return True
        if state[0] == state[4] == state[8] != E:
            return True
        if state[2] == state[4] == state[6] != E:
            return True
        return False
    
    def get_reward(self, state):
        # Reward R(s) for every possible state
        if state[0] == state[1] == state[2] != E:
            return 1 if state[0] == X else -1
        if state[3] == state[4] == state[5] != E:
            return 1 if state[0] == X else -1
        if state[6] == state[7] == state[8] != E:
            return 1 if state[0] == X else -1
        if state[0] == state[3] == state[6] != E:
            return 1 if state[0] == X else -1
        if state[1] == state[4] == state[7] != E:
            return 1 if state[0] == X else -1
        if state[2] == state[5] == state[8] != E:
            return 1 if state[0] == X else -1
        if state[0] == state[4] == state[8] != E:
            return 1 if state[0] == X else -1
        if state[2] == state[4] == state[6] != E:
            return 1 if state[0] == X else -1
        return False
        
    def get_transition_prob(self, action, new_state, old_state=None):
        if old_state is None:
            old_state = self.__state
        # returns the Transition Probability P(s'| s, a)
        # with s = old_state, a = action and s' = new_state

        # if the game is over, no transition can take place
        if self.is_done(old_state):
            return 0.0
        
        # the position of the action must be empty
        if old_state[action] != E:
            return 0.0
        
        # state after placing X
        state_after_X = copy(old_state)  # avoid unwanted changed by reference
        state_after_X[action] = X

        # check if game is done
        if self.is_done(state_after_X) and state_after_X == new_state:
            return 1.0

        # game is not done: calculate all possible states of the opponent
        possible_new_states = []
        possible_opponent_actions = self.get_possible_actions(state_after_X)
        for action in possible_opponent_actions:
            possible_new_state = copy(state_after_X)
            possible_new_state[action] = O
            possible_new_states.append(possible_new_state)
        if new_state not in possible_new_states:
            return 0.0
        
        # transition is possible, apply strategy:
        # random opponent, probability is 1 / (# of E before placing the new O)
        prob = 1 / (len(possible_new_states))
        return prob

In [2]:
# example of creation of an environment in the default state
mdp = GameEnvironment()
mdp.reset()
mdp.render()
state, done, reward, info = mdp.step(4)
print('state =', state, ', reward =', reward, ', done =', done)
mdp.render()
print('possible (internal) game states:')
mdp.get_possible_states()

   │   │   
───┼───┼───
   │   │   
───┼───┼───
   │   │   
state = [' ', 'O', ' ', ' ', 'X', ' ', ' ', ' ', ' '] , reward = False , done = False
   │ O │   
───┼───┼───
   │ X │   
───┼───┼───
   │   │   
possible (internal) game states:


[['X', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['X', 'O', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['X', 'O', 'X', ' ', ' ', ' ', ' ', ' ', ' '],
 ['X', 'O', 'X', 'O', ' ', ' ', ' ', ' ', ' '],
 ['X', 'O', 'X', 'O', 'X', ' ', ' ', ' ', ' '],
 ['X', 'O', 'X', 'O', 'X', 'O', ' ', ' ', ' '],
 ['X', 'O', 'X', 'O', 'X', 'O', 'X', ' ', ' '],
 ['X', 'O', 'X', 'O', 'X', 'O', ' ', 'X', ' '],
 ['X', 'O', 'X', 'O', 'X', 'O', 'O', 'X', ' '],
 ['X', 'O', 'X', 'O', 'X', 'O', 'O', 'X', 'X'],
 ['X', 'O', 'X', 'O', 'X', 'O', ' ', 'X', 'O'],
 ['X', 'O', 'X', 'O', 'X', 'O', 'X', 'X', 'O'],
 ['X', 'O', 'X', 'O', 'X', 'O', ' ', ' ', 'X'],
 ['X', 'O', 'X', 'O', 'X', ' ', 'O', ' ', ' '],
 ['X', 'O', 'X', 'O', 'X', 'X', 'O', ' ', ' '],
 ['X', 'O', 'X', 'O', 'X', 'X', 'O', 'O', ' '],
 ['X', 'O', 'X', 'O', 'X', 'X', 'O', 'O', 'X'],
 ['X', 'O', 'X', 'O', 'X', 'X', 'O', ' ', 'O'],
 ['X', 'O', 'X', 'O', 'X', 'X', 'O', 'X', 'O'],
 ['X', 'O', 'X', 'O', 'X', ' ', 'O', 'X', ' '],
 ['X', 'O', 'X', 'O', 'X', 'O', 'O', 'X'

In [3]:
mdp = GameEnvironment(['X', ' ', ' ', 
                               'O', 'X', ' ', 
                               ' ', ' ', 'O'])
mdp.render()
possible_actions = mdp.get_possible_actions()
print('possible actions: ', possible_actions)
random_agent_action = choice(possible_actions)
new_state, done, reward, info = mdp.step(random_agent_action)
mdp.render()
possible_actions = mdp.get_possible_actions()
print('possible actions: ', possible_actions)

 X │   │   
───┼───┼───
 O │ X │   
───┼───┼───
   │   │ O 
possible actions:  [1, 2, 5, 6, 7]
 X │ X │   
───┼───┼───
 O │ X │ O 
───┼───┼───
   │   │ O 
possible actions:  [2, 6, 7]
